# Exploring Mental Health Data
**Objective:** Predict whether an individual suffers from depression based on a set of responses from a mental health survey.

**Problem task:** Binary classification on the target variable depression (0 = false, 1 = true)

**Dataset source:** Kaggle - Playground Series S4E11


In [ ]:
#Marta path:
#Ricardo path:
#Sara path: "/Users/saracortez/feup/3o ano/iart/exploring_mental_health_data/data/train.csv"
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
train_data = pd.read_csv("/Users/saracortez/feup/3o ano/iart/exploring_mental_health_data/data/train.csv")
test_data = pd.read_csv("/Users/saracortez/feup/3o ano/iart/exploring_mental_health_data/data/test.csv")

print(train_data.head())
train_data.describe()

In [ ]:
#duplicate removal
bf = len(train_data)
print(f"Number of rows before removing duplicates: {len(train_data)}")
train_data = train_data.drop_duplicates()
af = len(train_data)
print(f"Number of rows after removing duplicates: {len(train_data)}")
if (bf-af) == 0:
    print("(No dup data found)")

In [ ]:
#missing value check
def missing_value_info(df):
    total = df.isnull().sum()
    percent = (total / len(df)) * 100
    return pd.DataFrame({'Missing Values': total, 'Percent Missing': percent}).sort_values(by='Percent Missing', ascending=False)
missing_info_with_0 = missing_value_info(train_data)
missing_info = missing_info_with_0[missing_info_with_0['Percent Missing'] > 0.0]
print(missing_info)
print(missing_info_with_0)


since these columns were spotted for missing values, we want to understand their appearance: how many are missing (NaN count in value_counts), if there unexpected 0s or negative values.

### Data prepp

In [ ]:
#train_data['Study Satisfaction'].value_counts(dropna=False)
#train_data['Academic Pressure'].value_counts(dropna=False)
#train_data['CGPA'].value_counts(dropna=False)
#train_data['Profession'].value_counts(dropna=False)
#train_data['Work Pressure'].value_counts(dropna=False)
#train_data['Job Satisfaction'].value_counts(dropna=False)
#train_data['Dietary Habits'].value_counts(dropna=False)
#train_data['Financial Stress'].value_counts(dropna=False)
pd.set_option('display.max_rows', None)
train_data['Degree'].value_counts(dropna=False)
#regulated
#Comclusion: all our missing vals are NANS


In [ ]:
# Forçar a visualização completa das contagens
pd.set_option('display.max_rows', None)  # Isso vai permitir que todos os valores sejam exibidos
print(train_data['Profession'].value_counts(dropna=False))


In [ ]:
train_data['Profession'].value_counts(dropna=False)

valid_professions = [
    "Teacher", "Content Writer", "Architect", "Consultant", "HR Manager",
    "Pharmacist", "Doctor", "Business Analyst", "Entrepreneur", "Chemist",
    "Chef", "Educational Consultant", "Data Scientist", "Researcher", "Lawyer",
    "Customer Support", "Marketing Manager", "Pilot", "Travel Consultant",
    "Plumber", "Sales Executive", "Manager", "Judge", "Electrician",
    "Financial Analyst", "Software Engineer", "Civil Engineer", "UX/UI Designer",
    "Digital Marketer", "Accountant", "Mechanical Engineer", "Graphic Designer",
    "Research Analyst", "Investment Banker", "Analyst", "Academic", "Unemployed", "Medical Doctor", "City Manager", "Family Consultant"
]

# corrigir erros digitação
def correct_profession(value):
    corrections = {
        "Finanancial Analyst": "Financial Analyst",
        # Adicionar mais ??
    }
    return corrections.get(value, value)

train_data['Profession'] = train_data['Profession'].apply(correct_profession)

test_data['Profession'] = test_data['Profession'].apply(correct_profession)

def clean_profession(value):
    if pd.isna(value):
        return value  # mantém NaN
    return value if value in valid_professions else "other"

train_data['Profession'] = train_data['Profession'].apply(clean_profession)
test_data['Profession'] = test_data['Profession'].apply(clean_profession)

train_data['Profession'].value_counts(dropna=False)

test_data['Profession'].value_counts(dropna=False)

# these are names: ["Yogesh", "Pranav", "Dev", "Yuvraj"]
# these seem to be localities ["Patna", "Visakhapatnam", "Nagpur", "FamilyVirar"]
# and these ? what are hey ? not jobs. ["Patna", "Visakhapatnam", "Nagpur", "FamilyVirar"]
#degrees like MBA
#substringing

In [ ]:
import re
# handle ranges like 6-8, handle more than/less than X,handle direct numeric values
print(len(train_data['Sleep Duration']))
#15 rows lost
def normalize_sleep_duration(column):
    def normalize(value):
        value = str(value).strip()

        match_range = re.match(r"(\d+)\s*-\s*(\d+)", value)
        if match_range:
            x, y = map(int, match_range.groups())
            return (x + y) / 2

        match_more = re.match(r"More than (\d+)", value, re.IGNORECASE)
        if match_more:
            return int(match_more.group(1)) + 0.5

        match_less = re.match(r"Less than (\d+)", value, re.IGNORECASE)
        if match_less:
            return int(match_less.group(1)) - 0.5

        try:
            return float(value)
        except ValueError:
            return pd.NA 

    return column.apply(normalize)

def normalize_large_sleep_values(column):
    def adjust_large(value):
        try:
            if pd.notna(value) and value >= 12:
                return round(value / 7 * 2) / 2
            return value
        except:
            return pd.NA
    return column.apply(adjust_large)

train_data['Sleep Duration'] = normalize_sleep_duration(train_data['Sleep Duration'])
train_data['Sleep Duration'] = normalize_large_sleep_values(train_data['Sleep Duration'])
test_data['Sleep Duration'] = normalize_sleep_duration(test_data['Sleep Duration'])
test_data['Sleep Duration'] = normalize_large_sleep_values(test_data['Sleep Duration'])

print(train_data['Sleep Duration'].value_counts())

In [ ]:
valid_dietary = ["Moderate", "Unhealthy", "Healthy"]

train_data["Dietary Habits"] = train_data["Dietary Habits"].apply(
    lambda x: x if pd.isna(x) or x in valid_dietary else "other"
)
test_data["Dietary Habits"] = test_data["Dietary Habits"].apply(
    lambda x: x if pd.isna(x) or x in valid_dietary else "other"
)


print(train_data['Dietary Habits'].value_counts(dropna = False))

In [ ]:
def normalize_degree(column):
    def clean(item):
        if isinstance(item, str):
            item = item.replace('.', '').replace(' ', '')
            return item
        else:
            return 'invalid'
    
    column = column.apply(clean)
    def remove_names(item):
            if (len(item) > 1 and item[0].isupper() and item[1].isupper() and item[0] in ['L', 'P', 'B', 'M']) or item == 'Class12' or item == "PhD":
                return item
            else:
                return 'invalid' 
    return column.apply(remove_names)
#importante dar NA aos inválidos para dar drop
train_data['Degree'] = normalize_degree(train_data['Degree'])
test_data['Degree'] = normalize_degree(test_data['Degree'])

degree_counts = train_data['Degree'].value_counts()
rare_degrees = degree_counts[degree_counts <= 5].index

train_data['Degree'] = train_data['Degree'].apply(lambda x: 'other' if x in rare_degrees else x)
test_data['Degree'] = test_data['Degree'].apply(lambda x: 'other' if x in rare_degrees else x)
     

print(train_data['Degree'].value_counts(dropna = False))

In [ ]:
print(train_data['Work/Study Hours'].value_counts(dropna=False))
"""
train_data['Work/Study Hours'].value_counts(dropna=False)
train_data['Academic Pressure'].value_counts(dropna=False)
train_data['CGPA'].value_counts(dropna=False)
train_data['Profession'].value_counts(dropna=False)
train_data['Work Pressure'].value_counts(dropna=False)
train_data['Job Satisfaction'].value_counts(dropna=False)
train_data['Dietary Habits'].value_counts(dropna=False)
train_data['Financial Stress'].value_counts(dropna=False)
train_data['Degree'].value_counts(dropna=False)"""



Our decision tree classifier requires binary values. Thus, let's convert bicategorical variables in to 0/1.

In [ ]:

# Gender column: Male -> 1, Female -> 0
train_data['Gender'] = train_data['Gender'].replace({'Male': 1, 'Female': 0})

# Working Professional or Student column: Working Professional -> 1, Student -> 0
train_data['Working Professional or Student'] = train_data['Working Professional or Student'].replace({'Working Professional': 1, 'Student': 0})

# Have you ever had suicidal thoughts?
train_data['Have you ever had suicidal thoughts ?'] = train_data['Have you ever had suicidal thoughts ?'].replace({'Yes': 1, 'No': 0})

train_data['Family History of Mental Illness'] = train_data['Family History of Mental Illness'].replace({'Yes': 1, 'No': 0})

test_data['Gender'] = test_data['Gender'].replace({'Male': 1, 'Female': 0})

test_data['Working Professional or Student'] = test_data['Working Professional or Student'].replace({'Working Professional': 1, 'Student': 0})

test_data['Have you ever had suicidal thoughts ?'] = test_data['Have you ever had suicidal thoughts ?'].replace({'Yes': 1, 'No': 0})

test_data['Family History of Mental Illness'] = test_data['Family History of Mental Illness'].replace({'Yes': 1, 'No': 0})

## Merging Columns

we can detect in our data two categories of individuals, identifiable by their attributes:
- **Students:** academic pressure, CGPA, study satisfaction, degree
- **Worker Professionals:** work pressure, profession, job satisfaction

So we decided to merge the columns that mean the same thing relatively yo each of these categories.

For example: 'Job Satisfaction' and 'Study Satisfaction' merge into just 'Satisfaction'

In [ ]:
train_data['Satisfaction'] = train_data[['Job Satisfaction', 'Study Satisfaction']].mean(axis=1, skipna=True)

train_data = train_data.drop(columns=['Job Satisfaction', 'Study Satisfaction'])
train_data['Pressure'] = train_data[['Work Pressure', 'Academic Pressure']].mean(axis=1, skipna=True)

train_data = train_data.drop(columns=['Work Pressure', 'Academic Pressure'])
print(train_data[['Pressure']].head())

print(train_data[['Satisfaction']].head())

train_data.loc[
    train_data['Profession'].isna() & (train_data['Working Professional or Student'] == 0),
    'Profession'
] = 'Student'

train_data.loc[
    train_data['Profession'].isna() & (train_data['Working Professional or Student'] != 0),
    'Profession'
] = 'other'
train_data = train_data.drop(columns=['CGPA'])
print(train_data[['Profession']].head())

test_data['Satisfaction'] = test_data[['Job Satisfaction', 'Study Satisfaction']].mean(axis=1, skipna=True)

test_data = test_data.drop(columns=['Job Satisfaction', 'Study Satisfaction'])
test_data['Pressure'] = test_data[['Work Pressure', 'Academic Pressure']].mean(axis=1, skipna=True)

test_data = test_data.drop(columns=['Work Pressure', 'Academic Pressure'])


test_data.loc[
    test_data['Profession'].isna() & (test_data['Working Professional or Student'] == 0),
    'Profession'
] = 'Student'

test_data.loc[
    test_data['Profession'].isna() & (test_data['Working Professional or Student'] != 0),
    'Profession'
] = 'other'
test_data = test_data.drop(columns=['CGPA'])


In [ ]:
train_data.to_csv("train_merge.csv", index=False)
test_data.to_csv("test_merge.csv", index=False)

##  Undersampling with priority

In [ ]:
#UNDERSAMPLING COM PRIORIDADE

from sklearn.utils import resample
import pandas as pd

X = train_data.drop("Depression", axis=1)
y = train_data["Depression"]

data = pd.concat([X, y], axis=1)

class_counts = data["Depression"].value_counts()
min_class_size = class_counts.min()

priority_cols = ["Profession", "Degree", "Dietary Habits"]

balanced_data = []

for label in class_counts.index:
    subset = data[data["Depression"] == label]

    if len(subset) > min_class_size:
        
        to_keep = min_class_size
        
        na_rows = subset[priority_cols].isna().any(axis=1)
        subset = subset[~na_rows]

        # Primeira prioridade: linhas com "invalid"
        invalid_rows = subset[subset[priority_cols].isin(["invalid"]).any(axis=1)]
        subset = subset.drop(invalid_rows.index)

        # Segunda prioridade: linhas com "other"
        other_rows = subset[subset[priority_cols].isin(["other"]).any(axis=1)]
        subset = subset.drop(other_rows.index)

        remaining_needed = to_keep


        if len(subset) >= remaining_needed:
            to_sample = subset.sample(remaining_needed, random_state=42)
        else:
            # Remover todos os 'priority' e sortear os restantes
            rows_needed = remaining_needed - len(subset)

            if len(other_rows) >= rows_needed:
                to_sample = pd.concat([subset, other_rows.sample(rows_needed, random_state=42)])
            else:
                still_needed = rows_needed - len(other_rows)
                to_sample = pd.concat([
                    subset,
                    other_rows,
                    invalid_rows.sample(still_needed, random_state=42)
                ])
    else:

        to_sample = subset

    balanced_data.append(to_sample)

undersampled_data = pd.concat(balanced_data)

undersampled_data.to_csv("final_train_dataset.csv", index=False)


In [ ]:
train_data = undersampled_data.copy()

Após o undersampling por forma a normalizar a razão No depression/Depression nos dados de teste, o número de casos ficou:

No depression: 5084

Depression: 5143

In [ ]:
import pandas as pd
df = pd.read_csv("final_train_dataset.csv")
print(df.columns)
print(df.head())


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import numpy as np

data = df.copy()
data_test = test_data.copy()

categorical_cols_train = data.select_dtypes(include=['object']).columns


# transforma as colunas com "palavras" em numeros
label_encoders = {}
for col in categorical_cols_train:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col].astype(str))
    label_encoders[col] = le

for col in categorical_cols_train:
    if col in data_test.columns:
        le = label_encoders[col]
        data_test[col] = data_test[col].apply(lambda x: x if x in le.classes_ else 'unknown')

        if 'unknown' not in le.classes_:
            le.classes_ = np.append(le.classes_, 'unknown')

        data_test[col] = le.transform(data_test[col].astype(str))


X_train = data.drop(columns='Depression')
y_train = data['Depression']

X_test = data_test[X_train.columns]


# Tratar valores nulos (usar média quando tem valores nulos)
imputer = SimpleImputer(strategy='mean')
train_data_imputed = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)



#DUVIDA do que fazer aqui
#apaga as linhas do teste com coluna nao numericas
#mask = X_test.applymap(lambda x: isinstance(x, (int, float, np.number))).all(axis=1)
#X_test_numeric = X_test[mask]

#ALTERNATIVA:
# Converter todas as colunas para numérico
X_test = X_test.apply(pd.to_numeric, errors='coerce')

test_data_imputed = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)


drop_cols = ['id', 'Name', 'City']
train_data_clean = train_data_imputed.drop(columns=drop_cols, errors='ignore')
test_data_clean = test_data_imputed.drop(columns=drop_cols, errors='ignore')

# Dividir em treino e teste
X_trainSplit, X_testSplit, y_trainSplit, y_testSplit = train_test_split(train_data_clean, y_train, test_size=0.2, random_state=42)



## Support Vector Machine

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
import time


start_time = time.time()

# I have tried other kernels (linear, poly and sigmoid) and they all perform similarly, the sigmoid being the worst
model = make_pipeline(StandardScaler(), SVC(kernel='rbf', C=1, gamma='scale', probability=True))
model.fit(X_trainSplit, y_trainSplit)

end_time = time.time()
print(f"Time taken to train the model: {end_time - start_time:.2f} seconds") #when probability is true (to collect data), it takes ~5 times longer to train

y_pred = model.predict(X_testSplit)
conf_matrix = confusion_matrix(y_testSplit, y_pred)

print("Accuracy:", accuracy_score(y_testSplit, y_pred))

# Confusion matrix heatmap
plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Greens', 
            xticklabels=['No Depression', 'Depression'],
            yticklabels=['No Depression', 'Depression'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - SVM')
plt.tight_layout()
plt.show()

# ROC curve
y_pred_proba = model.predict_proba(X_testSplit)[:, 1]
fpr, tpr, thresholds = roc_curve(y_testSplit, y_pred_proba)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label='ROC curve (area = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='red', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic - SVM')
plt.legend(loc='lower right')
plt.grid()
plt.show()


print("\nClassification Report:\n", classification_report(y_testSplit, y_pred))



## Plotting

In [ ]:
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
sb.scatterplot(x='Work/Study Hours', y='Sleep Duration',
                hue='Have you ever had suicidal thoughts ?', data=train_data_clean)
plt.title('Work Hours vs. Sleep Duration')
plt.show()

ALGORITHMS

In [ ]:
sb.pairplot(train_data_clean[[
    'Satisfaction', 'Sleep Duration', 'Work/Study Hours',
    'Have you ever had suicidal thoughts ?'
]], hue='Have you ever had suicidal thoughts ?')
plt.show()

K-NEIGHBOUR

Para este algortimo é necessário que os dados estejam padronizados para que nenhuma variável influencie desproporcionalmente o cálculo da distância. Isso melhora a performance e a precisão do modelo.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Padronizar os dados (por tudo na mesma escala : nao pode idade(0-100) e genero(0 ou 1), é preciso escalar)

#Este é com o split para fazermos as métricas
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_trainSplit)
X_test_scaled = scaler.transform(X_testSplit)


# Este é para os dados de teste real que queremos tentar prever
scaler = StandardScaler()
X_train_scaled_real = scaler.fit_transform(train_data_imputed)
X_test_scaled_real = scaler.transform(test_data_imputed)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

start_time = time.time()
# Instanciar e treinar o modelo
knn = KNeighborsClassifier(n_neighbors=6)
knn.fit(X_train_scaled, y_trainSplit)

end_time = time.time()
print(f"Time taken to train the model: {end_time - start_time:.2f} seconds") #when probability is true (to collect data), it takes ~5 times longer to train


# Fazer previsões
y_pred = knn.predict(X_test_scaled)

# Avaliar o desempenho
accuracy = accuracy_score(y_testSplit, y_pred)
conf_matrix = confusion_matrix(y_testSplit, y_pred)
class_report = classification_report(y_testSplit, y_pred)

class_report

Através da análise dos resultados obtidos concluímos que os valores de precision,  recall e F1-score indicam que o modelo tem boa capacidade de prever corretamente se o indivíduo tem ou não depressão, não havendo muitos falsos positivos nem falsos negativos e que não favorece uma mais classe mais do que a outra. Além disso, as médias macro e ponderada das métricas também ficaram em 91%, o que reforça o equilíbrio do desempenho do modelo.


graphics / confusion matrix

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import ConfusionMatrixDisplay, roc_curve, auc

# Confusion matrix heatmap
plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['No Depression', 'Depression'],
            yticklabels=['No Depression', 'Depression'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - KNN')
plt.tight_layout()
plt.show()

y_prob = knn.predict_proba(X_test_scaled)[:, 1]
fpr, tpr, thresholds = roc_curve(y_testSplit, y_prob)
roc_auc = auc(fpr, tpr)

# ROC Curve
plt.figure(figsize=(7, 5))
plt.plot(fpr, tpr, label=f'KNN (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - KNN')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


A confusion matrix e o gráfico da ROC curve vêm a reforçar aquilo que as metricas nos permitiram concluir. A confusion matrix permite-nos concluir o seguinte:

- Verdadeiros Positivos (TP): 4639 – Casos corretamente classificados como com depressão.

- Verdadeiros Negativos (TN): 4705 – Casos corretamente classificados como sem depressão.

- Falsos Positivos (FP): 379 – Casos sem depressão classificados erroneamente como com depressão.

- Falsos Negativos (FN): 504 – Casos com depressão classificados erroneamente como sem depressão.

Esses valores mostram que o modelo está bom tanto para identificar corretamente quem tem depressão quanto quem não tem. A maioria dos casos foi classificada corretamente.

A ROC curve mostra a relação entre a taxa de verdadeiros positivos e a taxa de falsos positivos. A Área sob a Curva (AUC) é 0.96, o que indica excelente capacidade discriminativa do modelo — ou seja, ele consegue distinguir muito bem entre as classes com e sem depressão.

Learning curve

In [ ]:
from sklearn.model_selection import learning_curve
from sklearn.model_selection import StratifiedKFold
import numpy as np

# Define o número de divisões de treino/teste
cv = StratifiedKFold(n_splits=5)

# Gera os valores de treino e validação para diferentes tamanhos de treino
train_sizes, train_scores, test_scores = learning_curve(
    knn, X_train_scaled, y_trainSplit, cv=cv,
    scoring='accuracy', train_sizes=np.linspace(0.1, 0.8, 5), n_jobs=1
)

# Calcula média e desvio padrão
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# Plot
plt.figure(figsize=(8, 5))
plt.plot(train_sizes, train_mean, 'o-', label='Training Accuracy')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.2)
plt.plot(train_sizes, test_mean, 'o-', label='Validation Accuracy')
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha=0.2)
plt.title('Learning Curve - KNN')
plt.xlabel('Training Set Size')
plt.ylabel('Accuracy')
plt.legend(loc='best')
plt.grid(True)
plt.tight_layout()
plt.show()


O modelo está a obter valores maior de accuracy com mais dados, tanto em treino quanto em validação.

Existe uma pequena diferença (~2%) entre treino e validação, o que é normal para KNN e indica leve overfitting.

As curvas não estão a convergir totalemnte totalmente o que pode indicar que o modelo ainda não atingiu o seu limite de aprendizado e que talvez, com mais dados ou com ajustes no modelo (tal como o valor de k), o desempenho de validação possa melhorar ainda mais.

KNN Accuracy para diferentes valores de k

In [ ]:
import numpy as np

# Testar vários valores de k (1 a 20)
k_values = range(1, 11)
accuracies = []

for k in k_values:
    knn_k = KNeighborsClassifier(n_neighbors=k)
    knn_k.fit(X_train_scaled, y_trainSplit)
    y_pred_k = knn_k.predict(X_test_scaled)
    acc = accuracy_score(y_testSplit, y_pred_k)
    accuracies.append(acc)

# Plot da acurácia vs. número de vizinhos (k)
plt.figure(figsize=(8, 5))
plt.plot(k_values, accuracies, marker='o')
plt.title('KNN Accuracy for Different Values of k')
plt.xlabel('Number of Neighbors (k)')
plt.ylabel('Accuracy')
plt.xticks(k_values)
plt.grid(True)
plt.tight_layout()
plt.show()


Através da análise do gráfico, podemos concluir que para valores de k muito baixos (como 1 ou 2), a acurácia também é mais reduzida. Isso pode ser explicado pelo fato de o modelo estar muito sensível ao overfitting, ajustando-se demais aos dados de treino.
A partir de k=3 o valor de accuracy aumenta significativamente e estabiliza a partir de k=5 , o que nos levou a utilizar este valor de k no nosso modelo. Contudo, o melhor desempenho está entre k = 9 e k = 10, com acurácia próxima de 0.917.

O modelo KNN melhora seu desempenho conforme k aumenta até cerca de 9 ou 10, o que indica que usar mais vizinhos ajuda a suavizar o efeito de possíveis outliers.

In [ ]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

# Calcular métricas individualmente
precision_0 = precision_score(y_testSplit, y_pred, pos_label=0)
recall_0 = recall_score(y_testSplit, y_pred, pos_label=0)
f1_0 = f1_score(y_testSplit, y_pred, pos_label=0)

precision_1 = precision_score(y_testSplit, y_pred, pos_label=1)
recall_1 = recall_score(y_testSplit, y_pred, pos_label=1)
f1_1 = f1_score(y_testSplit, y_pred, pos_label=1)

# Criar DataFrame com as métricas
metrics_df = pd.DataFrame({
    'Class': ['No Depression (0)', 'Depression (1)'],
    'Precision': [precision_0, precision_1],
    'Recall': [recall_0, recall_1],
    'F1-score': [f1_0, f1_1],
    'Support': [conf_matrix[0, 0] + conf_matrix[0, 1], conf_matrix[1, 0] + conf_matrix[1, 1]]
})

print(metrics_df)



In [ ]:
#Tentar adivinhar os reais

from sklearn.neighbors import KNeighborsClassifier

# Instanciar e treinar o modelo
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled_real, y_train)

# Fazer previsões
y_pred = knn.predict(X_test_scaled_real)

# Mostrar previsões
print("Previsões do modelo:")
print(y_pred)

Decison Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import time
clf = DecisionTreeClassifier()

start_time = time.time()

clf.fit(X_trainSplit, y_trainSplit)


end_time = time.time()
print(f"Time taken to train the Decision Tree Classifier: {end_time - start_time:.2f} seconds") #when probability is true (to collect data), it takes ~5 times longer to train

y_pred = clf.predict(X_testSplit)

cf_mx_clf = confusion_matrix(y_testSplit, y_pred)
plt.tight_layout()
plt.show()
accuracy = accuracy_score(y_testSplit, y_pred)
report = classification_report(y_testSplit, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:\n", report)
print("Confusion matrix\n", cf_mx_clf)


In [ ]:
plt.figure(figsize=(6, 5))
sns.heatmap(cf_mx_clf, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['No Depression', 'Depression'],
            yticklabels=['No Depression', 'Depression'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - Decision Tree Classifier')
plt.tight_layout()
plt.show()

y_prob = clf.predict_proba(X_test_scaled)[:, 1]
fpr, tpr, thresholds = roc_curve(y_testSplit, y_prob)
roc_auc = auc(fpr, tpr)

# ROC Curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='purple', label='ROC curve (area = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='red', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic - Decision Tree Classifier')
plt.legend(loc='lower right')
plt.grid()
plt.show()

Relembremos o número de casos da amostra:

No depression: 5084

Depression: 5143

A matriz de confusão permite-nos concluir o seguinte:

- Verdadeiros Positivos (TP): 4526 – 88%

- Verdadeiros Negativos (TN): 4452 - 88%

- Falsos Positivos (FP): 632 – 12%

- Falsos Negativos (FN): 617 – 12%

O modelo está bom tanto para identificar corretamente quem tem depressão quanto quem não tem. A maioria dos casos foi classificada corretamente. Como esperávamos, o Decision Tree Classifier foi de todos o que produziu uma _accuracy_ mais baixa, consequência da sua simplicidade. Por outro lado, o tempo de treino do modelo fica nos 0.2 segundos, afirmando-se como a alternativa menos dispendiosa em termos computacionais

A ROC curve mostra a relação entre a taxa de verdadeiros positivos e a taxa de falsos positivos. A Área sob a Curva (AUC) é 0.88, traduzindo a _accuracy_ do modelo.

In [ ]:
from sklearn.model_selection import learning_curve
from sklearn.model_selection import StratifiedKFold
import numpy as np

cv = StratifiedKFold(n_splits=5)

train_sizes, train_scores, test_scores = learning_curve(
    clf, X_train_scaled, y_trainSplit, cv=cv,
    scoring='accuracy', train_sizes=np.linspace(0.1, 0.8, 5), n_jobs=1
)

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

plt.figure(figsize=(8, 5))
plt.plot(train_sizes, train_mean, 'o-', label='Training Accuracy')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.2)
plt.plot(train_sizes, test_mean, 'o-', label='Validation Accuracy')
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha=0.2)
plt.title('Learning Curve - Decision Tree Classifier')
plt.xlabel('Training Set Size')
plt.ylabel('Accuracy')
plt.legend(loc='best')
plt.grid(True)
plt.tight_layout()
plt.show()


considerações training e validation accuracy

In [ ]:
#previsão dos dados de teste
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()
clf.fit(X_trainSplit, y_trainSplit)

y_pred = clf.predict(X_testSplit)

print("Previsões:")
print(y_pred)


NEURAL NETWORK (MLP)

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

mlp = MLPClassifier(max_iter=1000)

start_time = time.time()
mlp.fit(X_trainSplit, y_trainSplit)

end_time = time.time()
print(f"Time taken to train the MultiLayer Perceptron: {end_time - start_time:.2f} seconds") #when probability is true (to collect data), it takes ~5 times longer to train

y_pred = mlp.predict(test_data_imputed)

cf_mx_mlp = confusion_matrix(y_testSplit, y_pred)
plt.tight_layout()
plt.show()
accuracy = accuracy_score(y_testSplit, y_pred)
report = classification_report(y_testSplit, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)
print("Confusion matrix\n", cf_mx_mlp)

In [ ]:
plt.figure(figsize=(6, 5))
sns.heatmap(cf_mx_mlp, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['No Depression', 'Depression'],
            yticklabels=['No Depression', 'Depression'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - MultiLayer Perceptron')
plt.tight_layout()
plt.show()

y_prob = mlp.predict_proba(X_test_scaled)[:, 1]
fpr, tpr, thresholds = roc_curve(y_testSplit, y_prob)
roc_auc = auc(fpr, tpr)

# ROC Curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='green', label='ROC curve (area = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='red', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic - MultiLayer Perceptron')
plt.legend(loc='lower right')
plt.grid()
plt.show()

 considerações matrix e roc


In [ ]:
from sklearn.model_selection import learning_curve
from sklearn.model_selection import StratifiedKFold
import numpy as np

cv = StratifiedKFold(n_splits=5)

train_sizes, train_scores, test_scores = learning_curve(
    mlp, X_train_scaled, y_trainSplit, cv=cv,
    scoring='accuracy', train_sizes=np.linspace(0.1, 0.8, 5), n_jobs=1
)

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

plt.figure(figsize=(8, 5))
plt.plot(train_sizes, train_mean, 'o-', label='Training Accuracy')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.2)
plt.plot(train_sizes, test_mean, 'o-', label='Validation Accuracy')
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha=0.2)
plt.title('Learning Curve - Multi Layer Perceptron')
plt.xlabel('Training Set Size')
plt.ylabel('Accuracy')
plt.legend(loc='best')
plt.grid(True)
plt.tight_layout()
plt.show()

 considerações training e validation accuracy


In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(max_iter=1000)

mlp.fit(X_train_scaled_real, y_trainSplit)

y_pred = mlp.predict(test_data_imputed)

print("Previsões:")
print(y_pred)